## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
#! pip install bayesian-optimization

In [2]:
#! pip install catboost

In [3]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [4]:
import os
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv.zip']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [5]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [6]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [7]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        |  7.76     |  2.987    |  4.773    |
|  2        |  6.433    |  1.941    |  4.493    |
|  3        |  8.072    |  2.082    |  5.99     |
|  4        |  8.05     |  2.994    |  5.056    |
|  5        |  9.056    |  3.0      |  6.056    |


Great, now let's print the best parameters and the associated maximized target.

In [8]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 6.0555152083113075}


9.055515208311308

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [9]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [10]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [11]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
DepTime,100000.0,1341.52388,476.378445,1.0,931.0,1330.0,1733.0,2534.0
Distance,100000.0,729.39716,574.616860,30.0,317.0,575.0,957.0,4962.0


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [12]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [13]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [14]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [15]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

In [16]:
full_df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,1019,8290,11387,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,136,2247,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103



Split the new full dataframe into X_train and X_test. 

In [17]:
# Here I am doing slightly different
# I am going to split the train data into train and test data (80-20) as I want to validate the model prediction at the end.
from sklearn.model_selection import train_test_split 
X = full_df[:train_df.shape[0]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# X_new_data will be treated as new data for prediction.
X_new_data = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [18]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [19]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [20]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=370, min_child_samples=3590 will be ignored. Current value: min_data_in_leaf=370
|  1        |  0.7186   |  0.006594 |  0.04423  |  9.09     |  3.59e+03 |  370.2    |  536.6    |
|  2        |  0.7251   |  0.01132  |  0.03559  |  20.29    |  835.7    |  733.1    |  2.708e+0 |
|  3        |  0.7254   |  0.006095 |  0.04302  |  61.87    |  2.692e+0 |  756.1    |  3.618e+0 |
|  4        |  0.7414   |  0.01424  |  0.02649  |  21.34    |  1.473e+0 |  1.88e+03 |  3.966e+0 |
|  5        |  0.7358   |  0.0225   |  0.03963  |  42.0     |  6.071e+0 |  896.4    |  2.079e+0 |
|  6        |  0.7185   |  0.002319 |  0.02075  |  54.12    |  6.97e+03 |  385.4    |  87.41    |
|  7        |  0.7415   |  0.04085  |  0.04897  |  48.26    |  8.379e+0 |  1.915e+0 |  877

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [21]:
lgbBO.max

{'target': 0.7421140206786517,
 'params': {'lambda_l1': 0.0414512432624737,
  'lambda_l2': 0.0015120603276998325,
  'max_depth': 31.771929131971923,
  'min_child_samples': 6042.948494960009,
  'min_data_in_leaf': 1476.9647935540422,
  'num_leaves': 1632.4613720492332}}

Review the process at each step by using the '.res[0]' function.

In [22]:
lgbBO.res[0]

{'target': 0.7185896990191596,
 'params': {'lambda_l1': 0.006593899207454169,
  'lambda_l2': 0.04423282239799171,
  'max_depth': 9.09019996785143,
  'min_child_samples': 3590.264194735114,
  'min_data_in_leaf': 370.2050123569801,
  'num_leaves': 536.614479466117}}

In [39]:
#for i, res in enumerate(lgbBO.res):
#    print(f"Iteration {i}: {res}")

 **<font color='blue'> We have found the best parameters to build the lightgbm model and validate this model.</font>** 

In [25]:
# Set the parameters for lightgbm with the best parameters from Bayesian Optimization
params_best = {'lambda_l1': 0.043516670511739075,
  'lambda_l2': 0.014437319843176906,
  'max_depth': 45,
  'min_child_samples': 8688.616754111028,
  'min_data_in_leaf': 1415,
  'num_leaves': 2342,
  "objective" : "binary",
  "metric" : "auc", 
  'is_unbalance': True,
  "num_threads" : 20,
  "learning_rate" : 0.03,
  "subsample_freq" : 5,
  "bagging_seed" : 42,
  "verbosity" : -1}

In [26]:
lgb_train = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
lgb_test = lightgbm.Dataset(X_test, label=y_test)

In [27]:
lgb = lightgbm.train(params_best,
                       lgb_train,
                       num_boost_round=1000,
                       early_stopping_rounds=100,
                       valid_sets=lgb_test)

[1]	valid_0's auc: 0.706464
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.7117
[3]	valid_0's auc: 0.714467
[4]	valid_0's auc: 0.716901
[5]	valid_0's auc: 0.718489
[6]	valid_0's auc: 0.719947
[7]	valid_0's auc: 0.720844
[8]	valid_0's auc: 0.72196
[9]	valid_0's auc: 0.722235
[10]	valid_0's auc: 0.722955
[11]	valid_0's auc: 0.723725
[12]	valid_0's auc: 0.72398
[13]	valid_0's auc: 0.724217
[14]	valid_0's auc: 0.724562
[15]	valid_0's auc: 0.724756
[16]	valid_0's auc: 0.724842
[17]	valid_0's auc: 0.725164
[18]	valid_0's auc: 0.725351
[19]	valid_0's auc: 0.725396
[20]	valid_0's auc: 0.725521
[21]	valid_0's auc: 0.725652
[22]	valid_0's auc: 0.72612
[23]	valid_0's auc: 0.726392
[24]	valid_0's auc: 0.726611
[25]	valid_0's auc: 0.726951
[26]	valid_0's auc: 0.727273
[27]	valid_0's auc: 0.72753
[28]	valid_0's auc: 0.727657
[29]	valid_0's auc: 0.727798
[30]	valid_0's auc: 0.728007
[31]	valid_0's auc: 0.728062
[32]	valid_0's auc: 0.728156
[33]	valid_0's auc: 0.728

[285]	valid_0's auc: 0.744758
[286]	valid_0's auc: 0.744837
[287]	valid_0's auc: 0.744858
[288]	valid_0's auc: 0.744887
[289]	valid_0's auc: 0.744889
[290]	valid_0's auc: 0.744862
[291]	valid_0's auc: 0.744864
[292]	valid_0's auc: 0.744866
[293]	valid_0's auc: 0.74488
[294]	valid_0's auc: 0.744863
[295]	valid_0's auc: 0.744848
[296]	valid_0's auc: 0.744889
[297]	valid_0's auc: 0.744834
[298]	valid_0's auc: 0.744844
[299]	valid_0's auc: 0.744858
[300]	valid_0's auc: 0.74483
[301]	valid_0's auc: 0.744848
[302]	valid_0's auc: 0.744868
[303]	valid_0's auc: 0.744896
[304]	valid_0's auc: 0.744999
[305]	valid_0's auc: 0.745
[306]	valid_0's auc: 0.745026
[307]	valid_0's auc: 0.745047
[308]	valid_0's auc: 0.745021
[309]	valid_0's auc: 0.745015
[310]	valid_0's auc: 0.745042
[311]	valid_0's auc: 0.745024
[312]	valid_0's auc: 0.745005
[313]	valid_0's auc: 0.744986
[314]	valid_0's auc: 0.74498
[315]	valid_0's auc: 0.745012
[316]	valid_0's auc: 0.744973
[317]	valid_0's auc: 0.744975
[318]	valid_0's 

[569]	valid_0's auc: 0.74598
[570]	valid_0's auc: 0.746009
[571]	valid_0's auc: 0.74604
[572]	valid_0's auc: 0.746054
[573]	valid_0's auc: 0.74605
[574]	valid_0's auc: 0.746061
[575]	valid_0's auc: 0.746064
[576]	valid_0's auc: 0.74608
[577]	valid_0's auc: 0.746085
[578]	valid_0's auc: 0.746084
[579]	valid_0's auc: 0.746061
[580]	valid_0's auc: 0.746073
[581]	valid_0's auc: 0.746091
[582]	valid_0's auc: 0.746114
[583]	valid_0's auc: 0.746082
[584]	valid_0's auc: 0.746087
[585]	valid_0's auc: 0.746056
[586]	valid_0's auc: 0.746096
[587]	valid_0's auc: 0.746079
[588]	valid_0's auc: 0.746069
[589]	valid_0's auc: 0.746047
[590]	valid_0's auc: 0.746059
[591]	valid_0's auc: 0.746077
[592]	valid_0's auc: 0.746109
[593]	valid_0's auc: 0.746149
[594]	valid_0's auc: 0.746147
[595]	valid_0's auc: 0.746152
[596]	valid_0's auc: 0.746145
[597]	valid_0's auc: 0.746188
[598]	valid_0's auc: 0.746148
[599]	valid_0's auc: 0.746155
[600]	valid_0's auc: 0.746162
[601]	valid_0's auc: 0.746167
[602]	valid_0'

[849]	valid_0's auc: 0.746486
[850]	valid_0's auc: 0.746473
[851]	valid_0's auc: 0.746514
[852]	valid_0's auc: 0.74653
[853]	valid_0's auc: 0.746514
[854]	valid_0's auc: 0.746504
[855]	valid_0's auc: 0.746466
Early stopping, best iteration is:
[755]	valid_0's auc: 0.746596


In [28]:
y_pred = lgb.predict(X_test)
y_pred_bool = []
for i in range(len(y_pred)):
    if y_pred[i]<0.5:
        y_pred_bool.append(0)
    else:
        y_pred_bool.append(1)

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_bool)

array([[11837,  4490],
       [ 1391,  2282]], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_bool)

0.70595

**<font color='blue'> We can apply the model to the 'new data' for prediction. </font>** 

In [31]:
y_new_pred = lgb.predict(X_new_data)
y_new = []
for i in range(len(y_new_pred)):
    if y_new_pred[i]<0.5:
        y_new.append(0)
    else:
        y_new.append(1)

In [38]:
print(sum(y_new), len(y_new), round(100*sum(y_new)/len(y_new)),'%')

34679 100000 35 %


 **<font color='blue'> Based on the prediction, we have 34679 flights out of 100000 departing with delayed 15 mins. The percentage is about 35%. </font>** 